In [4]:
!pip install scikit-optimize

In [3]:
"""
RandLA-Net full hyperparameter tuning pipeline with Bayesian Optimization
"""

import os
import pickle
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

# ==============================
# DATASET
# ==============================
class RandLANetDataset(Dataset):
    def __init__(self, data_folder):
        self.data = []
        for root, dirs, files in os.walk(data_folder):
            if 'pc.pickle' in files:
                pc_path = os.path.join(root, 'pc.pickle')
                try:
                    with open(pc_path, 'rb') as f:
                        pc = pickle.load(f)  # numpy array
                    self.data.append(pc)
                except Exception as e:
                    print(f"Failed to load {pc_path}: {e}")
        print(f"Total samples found: {len(self.data)}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pc = self.data[idx]
        coords_feats = pc[:, :6].astype(np.float32)  # xyz + intensity + others
        labels = pc[:, -1].astype(np.int64)          # last column as label
        return {
            "features": torch.tensor(coords_feats, dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.long)
        }

# ==============================
# COLLATE FUNCTION
# ==============================
def collate_fn(batch):
    return {
        "features": [b["features"] for b in batch],
        "labels": [b["labels"] for b in batch]
    }

# ==============================
# RANDLA-NET MODEL
# ==============================
class SharedMLP(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, 1)
        self.bn = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

class RandlaNet(nn.Module):
    def __init__(self, d_in=6, d_out=[32,64,128], num_classes=4, dropout=0.5):
        super().__init__()
        self.encoder = nn.ModuleList()
        current_d = d_in
        for d in d_out:
            self.encoder.append(SharedMLP(current_d, d))
            current_d = d
        self.decoder = nn.ModuleList()
        for i in range(len(d_out)-1, 0, -1):
            self.decoder.append(SharedMLP(d_out[i]+d_out[i-1], d_out[i-1]))
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Conv1d(d_out[0], 64, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.fc2 = nn.Conv1d(64, num_classes, 1)

    def forward(self, x):
        x = x.transpose(1,2)
        skips = []
        for layer in self.encoder:
            x = layer(x)
            skips.append(x)
        for i, layer in enumerate(self.decoder):
            skip = skips[-(i+2)]
            x = torch.cat([x, skip], dim=1)
            x = layer(x)
        x = self.dropout(x)
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.fc2(x)
        return x.transpose(1,2)

# ==============================
# FOCAL LOSS
# ==============================
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    def forward(self, inputs, targets):
        B,N,C = inputs.shape
        inputs = inputs.reshape(-1,C)
        targets = targets.reshape(-1)
        ce_loss = nn.functional.cross_entropy(inputs, targets, reduction='none')
        p = torch.exp(-ce_loss)
        loss = (1-p)**self.gamma * ce_loss
        if self.alpha is not None:
            alpha_t = self.alpha[targets]
            loss = alpha_t * loss
        return loss.mean()

# ==============================
# METRICS
# ==============================
def compute_metrics(preds, targets, num_classes=4):
    preds = preds.cpu().numpy()
    targets = targets.cpu().numpy()
    acc = 100*(preds==targets).sum()/targets.size
    ious = []
    for c in range(num_classes):
        pred_c = (preds==c)
        target_c = (targets==c)
        inter = (pred_c & target_c).sum()
        union = (pred_c | target_c).sum()
        ious.append(inter/union if union>0 else 1.0)
    return acc, np.mean(ious), ious

# ==============================
# TRAINING LOOP
# ==============================
def train_model(model, dataloader, epochs=5, device="cpu", num_classes=4, lr=0.001):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = FocalLoss(alpha=torch.ones(num_classes).to(device), gamma=2.0)

    all_miou = []

    for epoch in range(epochs):
        total_loss = 0
        all_acc = []
        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            optimizer.zero_grad()
            for features, labels in zip(batch["features"], batch["labels"]):
                features = features.unsqueeze(0).to(device)
                labels = labels.unsqueeze(0).to(device)
                outputs = model(features)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                preds = torch.argmax(outputs, dim=-1)
                acc, miou, _ = compute_metrics(preds, labels, num_classes)
                all_acc.append(acc)
                all_miou.append(miou)
        avg_acc = np.mean(all_acc)
        avg_miou = np.mean(all_miou)
        print(f"Loss: {total_loss/len(dataloader):.4f} | Acc: {avg_acc:.2f}% | mIoU: {avg_miou:.4f}")

    return avg_miou

# ==============================
# HYPERPARAMETER OPTIMIZATION
# ==============================
def optimize_hyperparams(dataset, num_classes=4, max_calls=15):
    space  = [
        Real(1e-4, 1e-2, "log-uniform", name='lr'),
        Real(0.1, 0.6, name='dropout'),
        Integer(16, 64, name='d1'),
        Integer(32, 128, name='d2'),
        Integer(64, 256, name='d3'),
        Integer(2, 6, name='batch_size')
    ]

    @use_named_args(space)
    def objective(**params):
        lr = params['lr']
        dropout = params['dropout']
        d1, d2, d3 = int(params['d1']), int(params['d2']), int(params['d3'])
        batch_size = int(params['batch_size'])
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=0)
        model = RandlaNet(d_in=6, d_out=[d1,d2,d3], num_classes=num_classes, dropout=dropout)
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"\nEvaluating: lr={lr:.5f}, dropout={dropout:.2f}, d_out=[{d1},{d2},{d3}], batch_size={batch_size}")
        avg_miou = train_model(model, loader, epochs=2, device=device, num_classes=num_classes, lr=lr)
        return -avg_miou  # minimize negative mIoU

    from skopt import gp_minimize
    res = gp_minimize(objective, dimensions=space, n_calls=max_calls, n_random_starts=3, acq_func="EI", random_state=42)
    print("\nBest hyperparameters found: ", res.x)
    print("Best validation mIoU: ", -res.fun)
    return res

# ==============================
# MAIN
# ==============================
if __name__ == "__main__":
    DATA_FOLDER = r"C:/Users/umair.muhammad/RandLA-Net-pytorch/data"
    dataset = RandLANetDataset(DATA_FOLDER)
    CLASSES = 4

    # ------------------------------
    # Run hyperparameter optimization
    # ------------------------------
    result = optimize_hyperparams(dataset, num_classes=CLASSES, max_calls=10)

    # ------------------------------
    # Train final model with best hyperparameters
    # ------------------------------
    best_lr, best_dropout, best_d1, best_d2, best_d3, best_batch = result.x
    best_d1, best_d2, best_d3, best_batch = int(best_d1), int(best_d2), int(best_d3), int(best_batch)

    final_model = RandlaNet(d_in=6, d_out=[best_d1,best_d2,best_d3], num_classes=CLASSES, dropout=best_dropout)
    final_loader = DataLoader(dataset, batch_size=best_batch, shuffle=True, collate_fn=collate_fn, num_workers=0)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    print("\nTraining final model with best hyperparameters...")
    train_model(final_model, final_loader, epochs=10, device=device, num_classes=CLASSES, lr=best_lr)

    # ------------------------------
    # Save the best model
    # ------------------------------
    save_path = "best_randla_model.pth"
    torch.save(final_model.state_dict(), save_path)
    print(f"✔ Best model saved at: {save_path}")


Total samples found: 15

Evaluating: lr=0.00392, dropout=0.19, d_out=[53,89,150], batch_size=2


Epoch 1/2: 100%|██████████| 8/8 [00:17<00:00,  2.13s/it]


Loss: 0.5677 | Acc: 82.39% | mIoU: 0.3903


Epoch 2/2: 100%|██████████| 8/8 [00:15<00:00,  1.96s/it]


Loss: 0.0985 | Acc: 98.22% | mIoU: 0.4929

Evaluating: lr=0.00083, dropout=0.27, d_out=[23,94,75], batch_size=5


...

Loss: 0.0751 | Acc: 98.22% | mIoU: 0.5849


Epoch 9/10: 100%|██████████| 8/8 [00:10<00:00,  1.37s/it]


Loss: 0.0671 | Acc: 98.22% | mIoU: 0.5861


Epoch 10/10: 100%|██████████| 8/8 [00:11<00:00,  1.38s/it]

Loss: 0.0653 | Acc: 98.22% | mIoU: 0.5870
✔ Best model saved at: best_randla_model.pth
